# Adaptive robust lot sizing

This tutorial shows how to construct and solve the Adaptive robust lot sizing optimization problem. The problem can be formulated as follows:
$$
\begin{align}
\min_{\mathbf{x},\mathbf{Y}}\; &\mathbf{c}^T\mathbf{x} + \sum_{i=1}^N\sum_{j=1}^N t_{i,j}y_{i,j}\nonumber \\

\text{s.t. } & \mathbf{d} \leq \sum_{i=1}^N \mathbf{y}_{i}^R - \sum_{i=1}^N \mathbf{y}_{i}^C + \mathbf{x} \nonumber \\

&\mathbf{0} \leq \mathbf{x}\leq \mathbf{k} \nonumber \\

&\mathbf{d}\in \left\{\mathbf{d}: \mathbf{0}\leq \mathbf{d}\leq \mathbf{d}_{max},\; \sum_{i=1}^N d_i \leq \Gamma\right\} \nonumber \\
\end{align}
$$

where $\mathbf{d}\in\mathbb{R}^N$ is the uncertain demand vector, $\mathbf{Y}\in\mathbb{R}^{N\times N}$ is the stock matrix, i.e. $y_{i,j}$ stock can be transported from location $i$ to location $j$ at cost $t_{i,j}$. $\mathbf{y}_i^R, \mathbf{y}_i^C$ denote the $i$-th row/column of $\mathbf{Y}$, respectively. The stock matrix $\mathbf{Y}$ depends on the demand $\mathbf{d}$ and can be written as
$$
\mathbf{Y} = \mathbf{Y^0} + \sum_{i=1}^{N}\mathbf{Y}_i^d d_i
$$
where $\mathbf{Y}^0, \left\{ \mathbf{Y}_i^d \right\}_i \in \mathbb{R}^{N\times N}$.

In [3]:
import cvxpy as cp
import numpy as np

import lropt
from lropt.robust_problem import RobustProblem
from lropt import UncertainParameter

np.random.seed(seed=1234)

We define the constants as shown below:

In [4]:
n = 30
c = 20
k = 20
dmax = 1
Gamma = 20*np.sqrt(n)
coordinates = 10*np.random.rand(2, n)
t = ((coordinates[[0]] - coordinates[[0]].T) ** 2
     + (coordinates[[1]] - coordinates[[1]].T) ** 2) ** 0.5

The problem is defined and solved as shown below. Note that $\mathbf{Y}$ is defined indirectly through $\mathbf{Y}^0, \left\{ \mathbf{Y}_i^d \right\}_i$, and is a function of the demand vector $\mathbf{d}$.

In [5]:
uncertainty_set = lropt.Ellipsoidal(rho=Gamma, p=1, lb=0, ub=dmax)
u = lropt.UncertainParameter(n, uncertainty_set=uncertainty_set)

# u = lropt.UncertainParameter(n,
#                                 uncertainty_set = lropt.Ellipsoidal(p=2, data =np.ones((10,n))))
# formulate cvxpy variable
L = cp.Variable()
x = cp.Variable(n)
y_0 = cp.Variable((n,n))
c = lropt.Parameter(n, data = 20*np.ones((10,n)))
c.value=20*np.ones(n)
y = {}
for i in range(n):
    y[i]={}
    for j in range(n):
        y[i][j] = cp.Variable(n)

cons_expr = [c@x + cp.sum(cp.multiply(t,y_0)) + u@cp.sum([cp.sum([t[i,j]*y[i][j] for j in range(n)]) for i in range(n)]) - L]
for i in range(n):
    cons_expr += [((np.eye(n)[i]) - y_0[:,i] - cp.sum([y[j][i] for j in range(n)]) + y_0[i] + cp.sum([y[i][j] for j in range(n)]))@u -x[i]  ]
    for j in range(n):
        cons_expr +=[-y_0[i][j] - y[i][j]@u]  

objective = cp.Minimize(L)
constraints = [cp.maximum(*cons_expr)<=0]
constraints += [x<=20, x>=0]
eval_exp = c@x + cp.multiply(t,y_0) + u@cp.sum([cp.sum([t[i,j]*y[i][j] for j in range(n)]) for i in range(n)])
# formulate Robust Problem
prob = lropt.RobustProblem(objective, constraints,eval_exp = eval_exp )
prob.solve()

/Users/irina.wang/Desktop/Princeton/Project2/cvxpy/cvxpy/problems/problem.py:164: UserWarning: Constraint #0 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/Users/irina.wang/Desktop/Princeton/Project2/cvxpy/cvxpy/problems/problem.py:164: UserWarning: Constraint #1 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/Users/irina.wang/anaconda3/envs/lropt_s/lib/python3.11/site-packages/scipy/sparse/_index.py:134: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray_sparse(i, j, x)


KeyboardInterrupt: 

In [58]:
x.value

array([ 5.03092920e-16,  6.04737415e-16,  3.93802706e-16,  1.72460016e-16,
        1.53440950e-15,  8.09947497e-16,  1.66516743e-15,  1.11829820e-15,
        5.21147456e-16,  4.52941990e-16,  6.71621629e-16,  1.68065779e-15,
        1.57670304e-16,  1.12033652e-15, -1.46478715e-16,  5.06392201e-16,
       -1.02986252e-16,  1.14387718e-15,  1.23013313e-15,  1.03202783e-15,
       -2.73783784e-18,  1.82845596e-16, -3.87693205e-16,  1.48997478e-15,
       -1.88623151e-16,  1.69698982e-15,  1.02530386e-15,  6.58673373e-16,
        1.67467719e-15,  1.60089732e-15])

In [33]:
x = cp.Variable(N)
d = UncertainParameter(N, uncertainty_set=uncertainty_set)
y_0 = cp.Variable((N,N))
y_d = [cp.Variable((N,N)) for _ in range(N)]
y = 0
e = np.eye(N)
for j in range(N):
    y += y_d[j]*(e[j]@d)
y += y_0

#cp.multiply is elementwise multiplication
# objective = cp.Minimize(cp.sum(c*x) + cp.sum(cp.multiply(t,y)))
# constraints = [
#                 d <= cp.sum(y, axis=0) - cp.sum(y, axis=1) + x,
#                 y >= 0,
#                 x >= 0,
#                 x <= k,
#               ]
# prob = RobustProblem(objective=objective, constraints=constraints)
# prob.solve()

In [35]:
objective = cp.Minimize(cp.sum(c*x))
constraints = [d@np.ones(N) <= 110,
                x >=0,
              ]
prob = RobustProblem(objective=objective, constraints=constraints)
prob.solve()

-1.844877606758542e-09